In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# DescriptionGen: SEO-optimized product decription generation for retail using LangChain 🦜🔗

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_attributes_to_text.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_attributes_to_text.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_attributes_to_text.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

This notebook demonstrates how to use [LangChain](https://python.langchain.com/docs/get_started/introduction.html) together with Vertex AI LLMs to solve a problem that many large retailers face: the automatic generation of informative, SEO-optimized, and potentially creative product descriptions and titles, based on product attributes or specifications (often provided by a supplier). The process of automated description generation can achieve significant cost and time savings.

This tutorial shows how to start with raw product attributes and metadata to generate full, accurate, SEO-optimized, and safe descriptions using LLMs. You will also learn how to validate the descriptions using LLMs. Additionally, you how to use retrieval-augmented generation to make your prompts even better through k-NN (k-nearest neighbor) embedding search. Finally, it shows how to adapt the descriptions to your brand's writing style, even if the style differs per product.

Notes on running this tutorial:

The free public dataset sample used for this demo (authorized for personal or business use) can be downloaded [here](https://data.world/promptcloud/product-details-on-flipkart-com).

### Objectives

In this tutorial, you will learn how to use LangChain with the PaLM API to generate your product descriptions from existing product attributes. You will work through the following examples:

* Zero-shot prompting of foundational models to generate descriptions
* Zero-shot prompting of parameter-efficient tuned foundational models on custom corpora
* Few-shot untuned (and tuned) prompting using Vertex AI Embeddings to retrieve similar examples to incorporate into the prompt. This is done locally in-memory for the tutorial but can be extended to use [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) (a managed, scalable vector database)
* Using prompt-engineered LLM zero-shot prompts to check the safety, veracity, and quality of generated product descriptions, and generate a reasoning for its evaluation
* Evaluating the batch quality of generated prompts using n-gram overlap metrics like BLEU and ROUGE, as well as semantic similarity checks (using embeddings with the PaLM API) with both possible and negative examples
* Using basic and advanced LangChain constructs such as prompt templates, LLM chains, sequential LLM chains (for sequential prompts with multiple inputs and outputs, with the output of one fed as input to another), k-NN retrievers, and custom LLM classes to use Vertex AI and LangChain.

You will also see that using few-shot prompting computed using k-NN Vertex AI LLM embedding based  semantic similarity can boost the performance metrics across BLEU/ROUGE and semantic similarity. Additionally, you can add additional product properties by utilizing a Vertex Generative AI image captioning service, which can then also add to the richness of the product description.

### Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK & Other dependencies

In [ ]:
!pip3 install --user --upgrade  pydantic==1.10.9 \
                                keras-nlp==0.5.2 \
                                tensorflow==2.12.0 \
                                scikit-learn==1.2.2 \
                                lark==1.1.5 \
                                langchain==0.0.207 \
                                google-cloud-aiplatform==1.27.0 \
                                rouge-score==0.1.2

Restart the kernel to re-load the packages you just installed. You may see a pop-up warning that you can simply close.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cells below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"

import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

### Import libraries

In [ ]:
import json
import pprint
import time
import warnings

import keras_nlp
import pandas as pd
import tensorflow as tf
from langchain.chains import LLMChain, SequentialChain
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.retrievers import KNNRetriever
from sklearn.model_selection import train_test_split
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel

warnings.filterwarnings("ignore")

### Helper functions/classes for the rest of the code
These functions & classes show you how you can create your own custom LLM models and embeddings (e.g. from tfhub) using base langchain constructs if needed to. Langchain also supports Vertex LLMs ( for generation and embedding) natively.

In [ ]:
REQUESTS_PER_MINUTE = 16
pp = pprint.PrettyPrinter(width=200)


# for creating dynamic fewshot based on embedding based kNN approach
def compute_fewshot(
    query,
    retriever,
    ixed_df,
    delimiter="\n",
    input_label="input:",
    output_label="output:",
):
    """
    Takes in a query, a langchain retriever object and
    a dataframe indexed on the product attributes, computes K nearest
    neighbours based on embedding semantic similarity of product descriptions.
    Then returns output as new-line delimited string of format key:value for
    product attributes: product description of semantically similar products
    to the original query.

    E.g. Query = Color:White
         Brand = Adidas

       Output of function:
       Input: Color:White \n Brand= Nike
       Output: These Nike sport shoes help you with your everyday run!

       Input: Color:Grey \n Brand= Adidas,
       Output: Helps you protect your heels while running!
    """
    results = list()
    for spec in retriever.get_relevant_documents(query)[:3]:
        results.append("{}{}{}".format(input_label, delimiter, spec.page_content))
        results.append(
            "%s%s%s"
            % (output_label, delimiter, ixed_df.loc[spec.page_content]["description"])
        )
    return "\n".join(results)


# to & extract parse the various fields in a clean and uniform key:value format
def extract_tags(x, delimiter=":"):
    """
    Takes in a row of a dataframe, & extracts/ parses the various fields to
    create a newline delimited array of key:value pairs where key is the
    name of the product attribute: and value is the value of the attribute itself

    E.g. output
       Color: White
       Discounted_price: 200
    """
    results = list()
    name = x["product_name"]
    brand = x["brand"]
    price = x["discounted_price"]
    category = x["product_category_tree"]
    for sym in ["[", "]", '"']:
        category = category.replace(sym, "")

    results.append("{}{}{}".format("Product name", delimiter, name))
    results.append("{}{}{}".format("brand", delimiter, brand))
    results.append("{}{}{}".format("discounted price", delimiter, price))
    results.append("{}{}{}".format("Product category", delimiter, category))
    x = x["product_specifications"]

    if "nil" in x:
        return ""
    x = json.loads(x.replace("=>", ":"))["product_specification"]

    if type(x) is not list:
        results.append(
            "{}{}{}".format(x.get("key", "other detail"), delimiter, x.get("value", ""))
        )
    else:
        for attr in x:
            results.append(
                "%s%s%s"
                % (attr.get("key", "other detail"), delimiter, attr.get("value", ""))
            )

    return "\n".join(results)


# to compute the quality metrics of the generated text w.r.t reference text using Bleu, Rouge
# and semantic similarity scores
def compute_quality_metrics_batch(
    references, predictions, rouge_n_order=2, embedding=None
):
    """
    Takes a batch of generated text and corresponding reference texts,
    computes and prints their corresponding
    Bleu, Rouge and semantic similarity(using embeddings) scores.
    """
    rouge_n = keras_nlp.metrics.RougeN(order=rouge_n_order)
    bleu_n = keras_nlp.metrics.Bleu()
    rouge_scr = rouge_n(references, predictions)["f1_score"]
    bleu_scr = bleu_n(references, predictions)
    pp.pprint("Bleu:%s" % (bleu_scr.numpy()))
    pp.pprint("Rouge:%s" % (rouge_scr.numpy()))
    if embedding:
        embed_predictions = embedding.embed_documents(predictions)
        embed_references = embedding.embed_documents(references)
        m = tf.keras.metrics.CosineSimilarity(axis=1)
        m.update_state(embed_predictions, embed_references)
        pp.pprint("Semantic Similarity:%s" % (m.result().numpy()))


class VertexLLM(LLM):
    """
    Class to use Vertex AI LLMs to generate text throttled by specified
    rate to avoid quota errors.
    """

    model: TextGenerationModel
    predict_kwargs: dict

    def __init__(self, model, verbose, **predict_kwargs):
        super().__init__(model=model, verbose=verbose, predict_kwargs=predict_kwargs)

    @property
    def _llm_type(self):
        return "vertex"

    def _call(self, prompt, stop=None):
        result = self.model.predict(prompt, **self.predict_kwargs)
        return str(result)

    @property
    def _identifying_params(self):
        return {}


def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            # print(f'Sleeping {sleep_time:.1f} seconds')
            time.sleep(sleep_time)


class VertexEmbeddings(Embeddings):
    """
    Class to use Vertex AI LLMs to generate embeddings by specified
    rate to avoid quota errors.
    """

    def __init__(self, model, *, requests_per_minute=20):
        self.model = model
        self.requests_per_minute = requests_per_minute

    def embed_documents(self, texts):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches of 5 to stay below the quota limit
            head, docs = docs[:5], docs[5:]
            chunk = self.model.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

    def embed_query(self, text):
        single_result = self.embed_documents([text])
        return single_result[0]

## Data preparation
In this section, you will perform cleaning, parsing, preparing, and splitting of the full dataset. As part of the cleaning process, you will also ensure that the fields containing null or duplicate descriptions are filtered out.

In [ ]:
df = pd.read_csv(
    "gs://github-repo/use-cases/product_description_generation_retail/dataset_sample.csv"
)
df = (
    df[~(df["product_specifications"].str.contains("nil", na=False))]
    .dropna()
    .drop_duplicates(subset=["description"])
)
df.info()

In [ ]:
df["parsed_product_specs"] = df.apply(lambda x: extract_tags(x), axis=1)
df_processed = df[["parsed_product_specs", "description"]]

In [ ]:
# train/test split
train, test = train_test_split(df_processed, test_size=0.05, random_state=42)
print(len(train), len(test))

## Description generation using LLMs
This section shows how you can use zero-shot, few-shot, and parameter-efficient fine-tuned approaches to continuously improve on BLEU, Rouge, and semantic similarity metrics. It also shows how you can use LLMs to tag text for toxicity and assess their results using BLEU, Rouge, and semantic similarity scores.


### Import models

You load the pre-trained text generation model called `text-bison@001`.

In [ ]:
# You need to provide a GCP project here
embedding = VertexEmbeddings(
    TextEmbeddingModel.from_pretrained("textembedding-gecko@001"),
    requests_per_minute=REQUESTS_PER_MINUTE,
)

model = TextGenerationModel.from_pretrained("text-bison@001")
llm = VertexLLM(
    model, max_output_tokens=512, temperature=0.1, top_p=0.8, top_k=40, verbose=True
)

Define re-usable prompt templates from LangChain and make a sequential chain which first generates the description using the first prompt and then evalutes it for safety using the second one. The input variables would be filled in at runtime.

In [ ]:
template_descgen = """
Imagine you are a copywriter for the online ecommerce platform {platform_name}. You have been given a series of information about a product and are asked to write a short and truthful product description based on that information, that can be put on the website.
At the same time the description should be optimised for search engines, and look like a realistic description in freeform text for that product which could appear on the {platform_name}.com website. The information you have been given are the following:
{product_attributes}
"""
template_evaluate = """
Does this text look like a real human written description for a product which could be found on
{platform_name} and is also not toxic? Answer with one of the following options, along with a short explanation
Options:
- Yes
- No

text: {generated_description}
Option:
"""

prompt_descgen = PromptTemplate(
    input_variables=["platform_name", "product_attributes"],
    template=template_descgen,
)
prompt_eval = PromptTemplate(
    input_variables=["platform_name", "generated_description"],
    template=template_evaluate,
)
eval_chain = LLMChain(llm=llm, prompt=prompt_eval, output_key="is_safe")

### Method 1: Zero-shot description generation, validation & evaluation
To create a product description, you first need to create a prompt with placeholder variables for the product attributes. These variables will be filled in with the actual product attributes at runtime. You then need to attach corresponding large language models (LLMs) to the description generation and evaluation models. Finally, you need to chain the models together so that the product description generated by the first model is sent as an input to the second model. The outputs of both models will then be available upon execution of the SequentialChain.

In [ ]:
descgen_chain = LLMChain(
    llm=llm, prompt=prompt_descgen, output_key="generated_description"
)

overall_chain = SequentialChain(
    chains=[descgen_chain, eval_chain],
    input_variables=["platform_name", "product_attributes"],
    # Here you return multiple variables
    output_variables=["generated_description", "is_safe"],
    verbose=True,
)

In [ ]:
attrs = test["parsed_product_specs"].iloc[4]
orig_descr = test["description"].iloc[4]
pp.pprint("The original description:\n" + orig_descr)

In [ ]:
result_0shot_untuned = overall_chain(
    {"platform_name": "Flipkart", "product_attributes": attrs}
)
pp.pprint(result_0shot_untuned)

You see that the generated descriptions look SEO optimized and convincing, incorporating a lot of the product attributes.

Now you can evaluate the result of the above generated description against the original description:

In [ ]:
compute_quality_metrics_batch(
    [result_0shot_untuned["generated_description"]], [orig_descr], embedding=embedding
)

Now you can perform the same evaluation for a randomly sampled batch of 10 product specs and evaluate their LLM-generated descriptions and against their original descriptions. The output will be the average of the evaluation metrics across the 10 pairs.

In [ ]:
sample_test = test[["parsed_product_specs", "description"]].sample(10, random_state=42)
sample_test["generated_description_0shot"] = sample_test["parsed_product_specs"].apply(
    lambda x: overall_chain({"platform_name": "Flipkart", "product_attributes": x})[
        "generated_description"
    ]
)

sample_attrs = sample_test["parsed_product_specs"].values.tolist()
sample_descriptions = sample_test["description"].values.tolist()
sample_generated_descriptions_0shot = sample_test[
    "generated_description_0shot"
].values.tolist()

Now check the averaged output of the evaluation metrics:

In [ ]:
compute_quality_metrics_batch(
    sample_generated_descriptions_0shot, sample_descriptions, embedding=embedding
)

Now you can quickly skim through the original and generated descriptions and save the dataframe to disk in .CSV format

In [ ]:
sample_test[["description", "generated_description_0shot"]].head()

In [ ]:
sample_test.to_csv("./augmented_dataset.csv", index=False)

### Method 2: Few-shot description generation using dynamic k-nearest neighbours
In this section, you will use few-shot prompting to try to improve the LLM-generated descriptions compared to the zero-shot prompting technique you used earlier.

Here, instead of the few-shot examples being hardcoded or selected at random, the examples would be chosen by first embedding the query and document corpus, and then computing the k-nearest neighbors of the query embedding.

In [ ]:
template_descgen_fewshot = """
Imagine you are a copywriter for the online ecommerce platform {platform_name}. You have been given a series of information about a product as input and are asked to write a short and truthful product description based on that information as output, that can be put on the website.
At the same time the description should also be optimised for search engines and look like a realistic description for that product which could appear on the {platform_name}.com website.
{examples}
input:
{product_attributes}
output:
"""

prompt_descgen_fewshot = PromptTemplate(
    input_variables=["platform_name", "product_attributes", "examples"],
    template=template_descgen_fewshot,
)

descgen_chain_fewshot_untuned = LLMChain(
    llm=llm, prompt=prompt_descgen_fewshot, output_key="generated_description"
)

overall_chain_fewshot_untuned = SequentialChain(
    chains=[descgen_chain_fewshot_untuned, eval_chain],
    input_variables=["platform_name", "product_attributes", "examples"],
    # Here we return multiple variables
    output_variables=["generated_description", "is_safe"],
    verbose=True,
)

To create input/output examples to guide the model, you can use Vertex LLM embedding-based k-nearest neighbour computation based on input product specs in the train set. You then do nearest neighbour computation based on the product specs/attributes
in the train set and retrieve the corresponding description as well to guide the model through few-shot input/output examples.



In [ ]:
train_spec_ix = train.copy().set_index("parsed_product_specs")
retriever = KNNRetriever.from_texts(
    train["parsed_product_specs"].values.tolist()[:500], embedding
)
examples = compute_fewshot(attrs, retriever, train_spec_ix)

In [ ]:
result_fewshot_untuned = overall_chain_fewshot_untuned(
    {"platform_name": "Flipkart", "product_attributes": attrs, "examples": examples}
)
pp.pprint(result_fewshot_untuned["generated_description"])
pp.pprint(result_fewshot_untuned["is_safe"])

In [ ]:
sample_test["generated_description_fewshot"] = sample_test[
    "parsed_product_specs"
].apply(
    lambda x: overall_chain_fewshot_untuned(
        {
            "platform_name": "Flipkart",
            "product_attributes": x,
            "examples": compute_fewshot(x, retriever, train_spec_ix),
        }
    )["generated_description"]
)
sample_generated_descriptions_fewshot = sample_test[
    "generated_description_fewshot"
].values.tolist()

You should now see that all of the metrics (Bleu, Rouge, and Semantic Similarity) should have all improved -- and in some cases improved remarkably:

In [ ]:
compute_quality_metrics_batch(
    sample_generated_descriptions_fewshot, sample_descriptions, embedding=embedding
)

Now you can quickly skim through the original and generated descriptions and save the dataframe to disk in .CSV format

In [ ]:
sample_test[
    ["parsed_product_specs", "description", "generated_description_fewshot"]
].head()

In [ ]:
sample_test.to_csv("./augmented_dataset.csv", index=False)

### Method 3: Fine-tuned zero-shot description generation validation & evaluation

In this section you will perform parameter efficient fine tuning of the model on 500 randomly sampled (prompt, description) pairs from the training dataset, in order to tune the model to the description and writing style. Then you will generate descriptions for a batch of data and evaluate it against the original across the three metrics as demonstrated in previous sections.

<div class="alert alert-block alert-warning">
<b>⚠️ This section requires TPUs: Please note that fine tuning uses TPUs, hence you will need to ensure they are available to your project.</b>
</div>

In [ ]:
tuned_model = TextGenerationModel.from_pretrained("text-bison@001")

train_tuning = train.copy()

train_tuning["prompt_product_specs"] = train_tuning["parsed_product_specs"].apply(
    lambda x: prompt_descgen.format(platform_name="Flipkart", product_attributes=x)
)

train_tuning.rename(
    columns={"prompt_product_specs": "input_text", "description": "output_text"},
    inplace=True,
)

Note the code below will kickstart the tuning pipeline, which make take an hour or two to finish:

In [ ]:
tuned_model.tune_model(
    training_data=train_tuning.sample(10, random_state=42),
    train_steps=1,
    tuning_job_location="europe-west4",
    tuned_model_location="us-central1",
)

*Here* you load the most recently trained model and evaluate it on the same test sentences as before.

In [ ]:
model_id = tuned_model.list_tuned_model_names()[0]
tuned_model = TextGenerationModel.get_tuned_model(tuned_model_name=model_id)

In [ ]:
import datetime

print(datetime.datetime.now())  # started at 11:20am BST

In [ ]:
llm_tuned = VertexLLM(
    tuned_model,
    max_output_tokens=512,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

Create a new LLM chain with the same prompt template as before, just with the newly tuned model attached instead. Then include it in the sequential chain like you did with the zero-shot model, and then generate new descriptions for a batch of product attributes in the test set.

In [ ]:
descgen_chain_tuned = LLMChain(
    llm=llm_tuned, prompt=prompt_descgen, output_key="generated_description"
)

overall_chain_tuned = SequentialChain(
    chains=[descgen_chain_tuned, eval_chain],
    input_variables=["platform_name", "product_attributes"],
    # Here you return multiple variables
    output_variables=["generated_description", "is_safe"],
    verbose=True,
)

In [ ]:
result_0shot_tuned = overall_chain_tuned(
    {"platform_name": "Flipkart", "product_attributes": attrs}
)
pp.pprint(result_0shot_tuned)

In [ ]:
sample_test["generated_description_tuned_0shot"] = sample_test[
    "parsed_product_specs"
].apply(
    lambda x: overall_chain_tuned(
        {"platform_name": "Flipkart", "product_attributes": x}
    )["generated_description"]
)
sample_generated_descriptions_tuned_0shot = sample_test[
    "generated_description_tuned_0shot"
].values.tolist()

Compute batch Bleu, rouge and semantic similarity scores like before

In [ ]:
compute_quality_metrics_batch(
    sample_generated_descriptions_tuned_0shot, sample_descriptions, embedding=embedding
)

Now you can quickly skim through the original and generated descriptions and save the dataframe to disk in .CSV format

In [ ]:
sample_test[
    ["parsed_product_specs", "description", "generated_description_tuned_0shot"]
].head()

In [ ]:
sample_test.to_csv("./augmented_dataset.csv", index=False)

### Method 4: Few-shot description generation validation & evaluation using fine-tuned models

In this section, you perform parameter-efficient fine-tuning of the model on 500 randomly sampled (prompt, description) pairs from the training set in order to better match it to the description and writing style. Then, you generate descriptions for a batch and evaluate them against the original across the three metrics as done before.

Note: Since you did not train the tuned model in a few-shot fashion, sometimes it can be confused and generate unnatural text. In this case, you can rely on the validator model, which can catch this and be reused to reprompt until a valid reply is generated.

<div class="alert alert-block alert-warning">
<b>⚠️ This section requires TPUs: Please note that fine tuning uses TPUs, hence you will need to ensure they are available to your project.</b>
</div>

In [ ]:
descgen_chain_fewshot_tuned = LLMChain(
    llm=llm_tuned, prompt=prompt_descgen_fewshot, output_key="generated_description"
)
overall_chain_fewshot_tuned = SequentialChain(
    chains=[descgen_chain_fewshot_tuned, eval_chain],
    input_variables=["platform_name", "product_attributes", "examples"],
    # Here you return multiple variables
    output_variables=["generated_description", "is_safe"],
    verbose=True,
)

In [ ]:
result_fewshot_tuned = overall_chain_fewshot_tuned(
    {"platform_name": "Flipkart", "product_attributes": attrs, "examples": examples}
)
pp.pprint(result_fewshot_tuned["generated_description"])

It can happen sometimes that due to the reason mentioned above the fine tuned model generates incorrect responses: which is why the response of the validator model can be used to filter it out

In [ ]:
pp.pprint(result_fewshot_tuned["is_safe"])

In [ ]:
sample_test["generated_description_tuned_fewshot"] = sample_test[
    "parsed_product_specs"
].apply(
    lambda x: overall_chain_fewshot_tuned(
        {
            "platform_name": "Flipkart",
            "product_attributes": x,
            "examples": compute_fewshot(x, retriever, train_spec_ix),
        }
    )["generated_description"]
)
sample_generated_descriptions_tuned_fewshot = sample_test[
    "generated_description_tuned_fewshot"
].values.tolist()

In this case since the fine tuned model decreased the quality especially for Bleu score which has length penalty. Hence its important to monitor all 3 metrics

In [ ]:
compute_quality_metrics_batch(
    sample_generated_descriptions_tuned_fewshot,
    sample_descriptions,
    embedding=embedding,
)

Now you can quickly skim through the original and generated descriptions and save the dataframe to disk in .CSV format

In [ ]:
sample_test[
    ["parsed_product_specs", "description", "generated_description_tuned_fewshot"]
].head()

In [ ]:
sample_test.to_csv("./augmented_dataset.csv", index=False)

## Conclusion

This notebook demonstrates how SEO-optimized, truthful, and creative product descriptions can be created using Vertex AI Generative AI models and Langchain.

In this notebook, you learned how to:

* Leverage few-shot examples to ground the LLM and avoid hallucination, as well as tailor the generated descriptions closer to existing ones for similar products.
* Use the Vertex AI textembeddings model to assess semantic similarity
* Create LangChain Prompts, Retrievers, Chains, and Sequential Chains to generate more creative and engaging product descriptions.
* Batch evaluate the quality of generated text against original text using BLEU, ROUGE, and semantic similarity (based on cosine distance) scores.
* Guardrail the agent using a validator LLM model to ensure that the generated text is accurate, truthful, and creative.

### Possible next steps you can take:

* You can add additional product properties by utilizing the,[Vertex AI Image Captioning service](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning) which can then also add to the richness of the product description.
* You can try using RLHF (reinforcement learning with human feedback) to perform preference optimization between multiple generated descriptions.